# Modelling scheduling problem as constraint programming 

In this AIBT you learnt how to model a combinatorial optimisation problem using either Constraint Programming or Mixed Integer Linear programming paradigm. https://www.xoolive.org/optim4ai/
In this notebook you will be able to apply your CP modelling abilities to model RCPSP problem. Please refer to first notebook and slide deck [prez_Complex_Scheduling.pdf] to understand what constraints should be implemented

We will minizinc language directly in the notebook. In order to do that, just need to call this pip command.

In [ ]:
!pip install iminizinc

## Usefull imports

In [ ]:
import sys, os
this_folder = os.getcwd()
sys.path.append(os.path.join(this_folder, "discrete_optimisation/"))
import skdecide.hub

## Discrete optimisation imports : 
Let's load the same scheduling problem as the previous notebook : 

In [ ]:
from discrete_optimization.rcpsp.rcpsp_model import RCPSPModel, RCPSPSolution
from discrete_optimization.rcpsp.rcpsp_parser import files_available, parse_file

In [ ]:
file = [f for f in files_available if "j301_10.sm" in f][0]
model = parse_file(file)
print(model)

## CP modelling
In this section you will be invited to write a basic constraint programming model for RCPSP. You can do it directly in jupyter notebooks by using ```iminizinc```. 

In [ ]:
%load_ext iminizinc

Let's define then the necessary input data for our model. 

In [ ]:
# task_id -> index between 1..model.n_jobs
index_in_minizinc = {model.tasks_list[i]: i+1 for i in range(model.n_jobs)}
# upper bound on the makespan of the schedule.
max_time = model.horizon
# number of ressource
n_res = len(model.resources_list)
# capacity of the ressource
rc = [model.get_max_resource_capacity(r) for r in model.resources_list]
# number of tasks
n_tasks = model.n_jobs
# durations of the tasks
d = [model.mode_details[t][1]["duration"] for t in model.tasks_list]
# n_res*n_tasks array containing the ressource need for all tasks
rr = [[model.mode_details[t][1][r] for t in model.tasks_list] for r in model.resources_list]

# Adjacency matrix of the precedence graph : adj[i][j] = true if j is a successor of i.
adj = [[False for t in model.tasks_list] for t in model.tasks_list]
for t in model.tasks_list:
    for s in model.successors[t]:
        adj[index_in_minizinc[t]-1][index_in_minizinc[s]-1] = True

The variables will be assignated in the following minizinc model that you will have to fill. Running this cell will run the model solver for 100 seconds using the Chuffed solver. 

<b>Exercise</b> : write down a basic model that provides feasible solution to the rcpsp problem. And if possible giving good objectives value.

In [ ]:
%%minizinc -m bind --v --time-limit 100000 --solver chuffed
int: max_time;
set of int: TIMES=0..max_time;
int: n_res;                     % The number of resources
set of int: Res = 1..n_res;     % The set of all resources
array [Res] of int: rc;         % The resource capabilities

% Tasks
%
int: n_tasks;                           % The number of tasks
set of int: Tasks = 1..n_tasks;         % The set of all tasks
array [Tasks] of int: d;                % The task durations
array [Res, Tasks] of int: rr ;         % The resource requirements
array [Tasks, Tasks] of bool: adj;      % Adjacency matrix
array [Tasks] of set of Tasks: suc = [{p|p in Tasks where adj[j,p]}|j in Tasks];

% Variable 
array [Tasks] of var TIMES: s;  % The start times
var TIMES: makespan      ;  % The project duration (makespan)

% WRITE YOUR CONSTRAINTS :) !!!


output ["starting time = \(s), makespan = \(makespan)"];
solve minimize makespan;

In [ ]:
print(s)

In [ ]:
#%%minizinc -m bind --verbose --solver chuffed
#%load correction/nb2_rcpsp_mzn.mzn  # Local correction, that you don't have !

# Solution checker 
You can rebuild a solution object from the obtained schedule. And check if it is satisfiable.

In [ ]:
solution = RCPSPSolution(problem=model, 
                         rcpsp_schedule={model.tasks_list[i]: {"start_time": s[i],
                                                               "end_time": s[i]+d[i]}
                                         for i in range(model.n_jobs)})
print("Satisfy : ", model.satisfy(solution))
print("Evaluation : ", model.evaluate(solution))

In [ ]:
from discrete_optimization.rcpsp.rcpsp_utils import plot_ressource_view, plot_task_gantt
plot_ressource_view(model, solution)
plot_task_gantt(model, solution)